<a href="https://colab.research.google.com/github/LevonaHayoun/3d_Unet_Segmentation/blob/main/2nifti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Converting DICOM images to 3D NIFTI image

In [ ]:
import os
import nibabel as nib
import pydicom
import numpy as np


def convert_dicom_to_nifti(dicom_dir, reference_affine):
    """Convert a series of DICOM images to a NIfTI image """

    dicom_files = [pydicom.dcmread(os.path.join(dicom_dir, f)) for f in os.listdir(dicom_dir) if f.endswith('.dcm')]
    dicom_files.sort(key=lambda x: float(x.ImagePositionPatient[2]))  # Sort by slice location

    pixel_data = np.stack([ds.pixel_array for ds in dicom_files], axis=-1)

    nifti_img = nib.Nifti1Image(pixel_data, reference_affine)

    return nifti_img


# Paths to the main directories
cases_dir = r'C:\Users\D_Learning\CASE'

# reference NIfTI image/
reference_nifti_path = r'C:\Users\D_Learning\ref_img.nii.gz'
reference_nifti_img = nib.load(reference_nifti_path)
reference_affine_img = reference_nifti_img.affine

reference_nifti_path = r'C:\Users\D_Learning\ref_lbl.nii.gz'
reference_nifti_lbl = nib.load(reference_nifti_path)
reference_affine_lbl = reference_nifti_lbl.affine

# Process each case
for case in os.listdir(cases_dir):
    case_path = os.path.join(cases_dir, case)

    # Ensure processing only directories named "case_{number}"
    if os.path.isdir(case_path) and case.startswith('case_'):

        images_dir = os.path.join(case_path,
                                 'DICOM')
        masks_dir = os.path.join(case_path,
                                 'MASK')


        img_nifti = convert_dicom_to_nifti(images_dir, reference_affine_img)
        img_output_path = os.path.join(case_path, f'img_{case}.nii.gz')
        nib.save(img_nifti, img_output_path)
        print(f'Image saved to {img_output_path}')

        # Convert DICOM masks to NIfTI
        lbl_nifti = convert_dicom_to_nifti(masks_dir, reference_affine_lbl)
        lbl_output_path = os.path.join(case_path, f'lbl_{case}.nii.gz')
        nib.save(lbl_nifti, lbl_output_path)
        print(f'Label saved to {lbl_output_path}')
